### MNIST with Estimators

In [10]:
import numpy as np
import tensorflow as tf
import argparse
import sys
import math
import gzip
import os
import shutil
from six.moves import urllib
from tensorflow.python.platform import gfile

In [4]:
#Enable Logging
tf.logging.set_verbosity(tf.logging.INFO)
# tf.logging.log(logging.INFO, "Tensorflow Version"+tf.__version__)

In [5]:
def maybe_download_and_extract(filename, work_directory, source_url):
    """
    To download &/ extract the zip files from a given working directory
    Args:
        filename: File to be extracted
        work_directory: Path to the directory
        source_url: URL to download the zip files
    Returns:
        unzipped_filepath: Path to the unzipped files
    """    
    if filename[-3:] == ".gz":
        unzipped_filename = filename[:-3]
    else:
        unzipped_filename = filename

    if not gfile.Exists(work_directory):
        gfile.MakeDirs(work_directory)

    filepath = os.path.join(work_directory, filename)
    unzipped_filepath = os.path.join(work_directory, unzipped_filename)

    if not gfile.Exists(unzipped_filepath):
        urllib.request.urlretrieve(source_url, filepath)

        if not filename == unzipped_filename:
            with gzip.open(filepath, 'rb') as f_in:
                with open(unzipped_filepath, 'wb') as f_out: # remove .gz
                    shutil.copyfileobj(f_in, f_out)

        with gfile.GFile(filepath) as f:
            size = f.size()
        print('Successfully downloaded and unzipped', filename, size, 'bytes.')
    return unzipped_filepath

In [6]:
def read_labels(tf_bytestring):
    """
    Decodes the labels in string format to Tensor
    Args:
        tf_string: Labels in string format
    Returns:
        labels: As tf.float32
    """
    label = tf.decode_raw(tf_bytestring, tf.uint8)
    return (tf.reshape(label, []))

In [7]:
def read_images(tf_bytestring):
    """
    Decodes the input data in string format to Tensor
    Args:
        tf_string: Images in string Format
    Returns:
        image: as tf.float32
    """
    image = tf.decode_raw(tf_bytestring, tf.uint8)
    return(tf.cast(image, tf.float32))

In [8]:
def load_data_mnist(data_dir):
    """
    May download and extract the data files
    Args:
        data_dir: Path to where dataset files are stored
    Returns:
        train_data:
        train_labels:
        test_data:
        test_labels:
    """
    train_images_file = 'train-images-idx3-ubyte.gz'
    local_train_images_file = maybe_download_and_extract(train_images_file, data_dir, SOURCE_URL+train_images_file)
    train_labels_file = 'train-labels-idx1-ubyte.gz'
    local_train_labels_file = maybe_download_and_extract(train_labels_file, data_dir, SOURCE_URL+train_labels_file)
    test_images_file = 't10k-images-idx3-ubyte.gz'
    local_test_images_file = maybe_download_and_extract(test_images_file, data_dir, SOURCE_URL+test_images_file)
    test_labels_file = 't10k-labels-idx1-ubyte.gz'
    local_test_labels_file = maybe_download_and_extract(test_labels_file, data_dir, SOURCE_URL+test_labels_file)
    return local_train_images_file, local_test_images_file, local_train_labels_file, local_test_labels_file

In [9]:
def load_data(imagefile, labelfile):
    """
    Creates interleaved images and labels dataset
    Args:
        imagefile: To create tensor of image_data
        labelfile: To create tensor of label_data
    Returns:
        dataset: the combined dataset of images and labels
    """
    image_data = tf.FixedLengthRecordDataset(imagefile, 28*28, header_bytes=16, buffer_size=1024*16).map(read_images)
    label_data = tf.FixedLengthRecordDataset(labelfile, 8, header_bytes=16, buffer_size=1024*16).map(read_labels)
    dataset = tf.data.Dataset.zip((image_data, label_data))
    return dataset

In [ ]:
def nodes_for_model(dataset):
    """
    For making the data appropriate to be passed to the model function
    Args:
        dataset:
    Returns:
    
    """
    features, labels = dataset.make_one_shot_iterator().get_next()
    return {'image': features}, labels

In [ ]:
def train_data_input_fn(train_images_file, train_labels_file):
    """
    Shuffle the data and pass it in the appropriate format
    Args:
        train_images_file: File with training image data
        train_labels_file: File with training label data
    Returns:
    
    """
    dataset = load_data(train_images_file, train_labels_file)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(60000)
    dataset = dataset.batch(100)
    return nodes_for_model(dataset)

In [ ]:
def eval_data_input_fn(test_images_file, test_labels_file):
    """
    Shuffle the data and pass it in the appropriate format
    Args:
        test_images_file: File with testing image data
        test_labels_file: File with testing label data
    Returns:
    
    """
    dataset = load_data(test_images_file, test_labels_file)
    dataset = dataset.batch(10000)
    #No need to repeat dataset for evaluation
    dataset = dataset.repeat(1)
    return nodes_for_model(dataset)

In [ ]:
def conv_model():
    input = tf.placeholder(tf.float32, [None, 28, 28, 1], name='input')
    labels = tf.placeholder(tf.int32, [None, 10])
    
    with tf.variable_scope('conv1') as scope:
        weights = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1), name=scope.name)
        conv = tf.nn.conv2d(input, weights, [1, 1, 1, 1], padding="SAME")
        biases = tf.Variable(tf.constant(0.1, shape=[32]))
        conv1 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope.name)
    
    pool1 = tf.nn.max_pool(conv1, strides=[1, 2, 2, 1], padding="SAME", ksize=[1, 2, 2, 1], name="pool1")

    with tf.variable_scope('conv2') as scope:
        weights = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1), name=scope.name)
        conv = tf.nn.conv2d(pool1, weights, [1, 1, 1, 1], padding="SAME")
        biases = tf.Variable(tf.constant(0.1, shape=[64]))
        conv2 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope.name)

    pool2 = tf.nn.max_pool(conv2, strides=[1, 2, 2, 1], padding="SAME", ksize=[1, 2, 2, 1], name="pool2")

    with tf.variable_scope('fc1') as scope:
        weights = tf.Variable(tf.truncated_normal([7*7*64, 200], stddev=0.1), name=scope.name)
        pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
        biases = tf.Variable(tf.constant(0.1, shape=[200]))
        fc1 = tf.nn.relu(tf.matmul(pool2_flat, weights)+biases, name=scope.name)

    dropout = tf.nn.dropout(fc1, 0.75)

    with tf.variable_scope('logits') as scope:
        weights = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1), name=scope.name)
        biases = tf.Variable(tf.constant(0.1, shape=[10]))
        logits = tf.nn.relu(tf.matmul(dropout, weights)+biases, name=scope.name)

In [ ]:
def main(argv):
    """
    """
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_dir", default="data", help="where training data will be unzipped")
    parser.add_argument("--work_dir", default="checkpoints", help="where training checkpoints will be stored")
    parser.add_argument("--hp_lr", default="0.0001", help="learning rate")
    args = parser.parse_args()
    arguments = args.__dict__
    #Make a dictionary of all arguments
    args_dict = {k: v for k,v in arguments.items()}
    data_dir = args_dict['data_dir']
    working_dir = args_dict['work_dir']
    learn_rate = args_dir['hp_lr']
    train_images_file, test_images_file, train_labels_file, test_labels_file = load_mnist_data(data_dir)
    #...........
    def train_input_fn(): 
        return train_data_input_fn(train_images_file, train_labels_file)
    def eval_input_fn():
        return eval_data_input_fn(test_images_file, test_labels_file)
    #.................
    training_config = tf.estimator.RunConfig(model_dir=working_dir, summary_steps=10, save_checkpoints_steps=200)
    estimator = tf.estimator.Estimator(model_fn=conv_model, model_dir=work_dir, params=learn_rate, config=training_config)
    train_spec = tf.estimator.TrainSpec(train_input_fn, max_steps=1000)
#     export_latest = tf.estimator.LatestExporter("mnist-model",serving_input_receiver_fn=serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(eval_input_fn, steps=1,  throttle_secs=60)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    